In [ ]:
import random
from queue import PriorityQueue

top_Row = [0, 6, 12, 18, 24, 30]  # used to compute the top row's search order
bottom_Row = [5, 11, 17, 23, 29, 35]  # used to compute the bottom row's search order
S_Node = 0    #start node (initialized to 0)
G_Node = 0     #Goal node  (initialized to 0)
system = []
ListOfNodes = []
ListOfChoices = []
ListOfHurdles = []
Maze = []

def DFS_algo():
    Stack_DFS = []
    Visited_DFS = []
    backMapDatabase = {}  # variable to keep track of all visited nodes from the goal node to the start node
    Stack_DFS.append(S_Node)  # Append the start node to the Stack_DFS
    
    #Inside the loop, it pops the top node from the stack and checks if it has been visited
    while True:
        activeNode = Stack_DFS.pop()  
        if activeNode in Visited_DFS:   
            continue
        else:
            Visited_DFS.append(activeNode)   #if a node not present in a visited list it appends

        if G_Node == activeNode:          #checking whehter the goal node is equal to the current node
            break
            
        #function calls the searching_order function to get the order in which the neighbors of the active 
        #node should be explored, and reverses the list
        StackTemp = searching_order(activeNode)
        StackRevTemp = StackTemp[::-1]

        V_Nodes = []      #list for Valid nodes
        for everyTemporaryNode in StackRevTemp:
            if everyTemporaryNode in ListOfHurdles:   #filtering the barriers and avoiding the barrier process
                continue
            else:
                V_Nodes.append(everyTemporaryNode)

        for everyCheckedNode in V_Nodes:
            if everyCheckedNode in Visited_DFS:
                continue
            else:
                Stack_DFS.append(everyCheckedNode)  #appends the remaining nodes to the Stack_DFS
                backMapDatabase[everyCheckedNode] = activeNode  #updates the backMapDatabase dictionary to store the
                                                                #path from the goal node to the start node

    shortestPath = SPF(backMapDatabase)     #defining the shortest path from the SPF (shortest path finder)function
    outcome("Depth First Search", Visited_DFS, shortestPath) #using the backMapDatabase dictionary

def chebyshev_distance_heuristic(startnode, goalnode):
    # Extract the coordinates of the node and the goal
    node_x, node_y = startnode // 6, startnode % 6
    goal_x, goal_y = goalnode // 6, goalnode % 6

    # Calculate the Chebyshev distance using the formula provided
    chebyshev_distance = max(abs(node_x - goal_x), abs(node_y - goal_y))
    return chebyshev_distance

def AStar_Search_algo():
    #initializes two dictionaries called Node_Cost and FS to store the cost of reaching each node from the start node
    #and the final cost of reaching the goal node from each node 
    FS = {cell: float('inf') for cell in system}  # FS = final cost which is the f(n)
    FS[S_Node] = chebyshev_distance_heuristic(S_Node, G_Node)
    Node_Cost = {cell: float('inf') for cell in system}  # g(n) = Node_Heuristic cost from node to goal node
    Node_Cost[S_Node] = 0

    Queue_Search = PriorityQueue()
    Queue_Search.put((chebyshev_distance_heuristic(S_Node, G_Node), #defining the cost of the start node and goal node from 
                      chebyshev_distance_heuristic(S_Node, G_Node), S_Node))  # the chebyshev method
    Path_Tracking = {}     #to store the path from the start node to the goal node
    list = []
    
    while not Queue_Search.empty():  #a loop, which continues until the Queue_Search is empty
        activeNode = Queue_Search.get()[2]   #gets the node with the lowest final cost from the Queue_Search
        list.append(activeNode)    #marks it as visited, and adds it to the list 
        if activeNode == G_Node:
            break
        tempQueue = searching_order(activeNode)

        Valid_Nodes = []
        for everyTemporaryNode in tempQueue:  #filters out the barrier nodes from the list. 
            if everyTemporaryNode in ListOfHurdles:
                continue
            else:
                Valid_Nodes.append(everyTemporaryNode)
                
        for d in Valid_Nodes:    #path tracking and chebyshev cost calculator 
            Node_Cost_temporary = Node_Cost[activeNode] + 1
            temporary_FS = Node_Cost_temporary + chebyshev_distance_heuristic(d, G_Node)
            if temporary_FS < FS[d]:#If the cost is lower than the current final cost for the node, it updates the Node_Cost
                Node_Cost[d] = Node_Cost_temporary
                FS[d] = temporary_FS
                Queue_Search.put((temporary_FS, chebyshev_distance_heuristic(d, G_Node), d))
                Path_Tracking[d] = activeNode

    shortestPath = SPF(Path_Tracking)    #defining the shortest path from the SPF (shortest path finder)function
    outcome("A-star Search", list, shortestPath)  #using the Path_Tracking dictionary

def searching_order(active_node):  # checking to see whether the active node is at the maze's edge
    tempQueue = []
    if active_node - 6 >= 0:
        tempQueue.append(active_node - 6)
    if active_node in top_Row:
        pass
    else:
        tempQueue.append(active_node - 1)
    if active_node in bottom_Row:
        pass
    else:
        tempQueue.append(active_node + 1)
    if active_node + 6 <= 35:
        tempQueue.append(active_node + 6)
    return tempQueue

def SPF(bm):  # following the track path and get the shortest path
    bp = {}     #backpath, a dictionary to hold the values of the back tracking process
    finder = G_Node
    while finder != S_Node:
        bp[bm[finder]] = finder
        finder = bm[finder]

    bpList = []
    bpList.append(G_Node)
    for count in bp:
        bpList.append(count)
    return bpList[::-1]


def maze():     #to generate maze
    global Maze, ListOfNodes, ListOfChoices, S_Node, G_Node, ListOfHurdles, system
    # maze=generated maze , NodeList = All the nodes , ChoiceList = Possible traverse nodes
    for i in range(36):
        Maze.append(i)
        ListOfNodes.append(i)
        ListOfChoices.append(i)
        system.append(i)
    #randomizing the start node and goal node
    S_Node = random.randint(0, 11)   
    G_Node = random.randint(24, 35)
    #UI Designing code block
    print('\033[1m' + "\nStart Node: " + '\033[0m', S_Node, '\033[1m' + " Goal Node: " + '\033[0m', G_Node)
    ListOfChoices.remove(S_Node)
    ListOfChoices.remove(G_Node)

    while len(ListOfHurdles) != 4:   #defining the four barriers
        barrier = random.choice(ListOfChoices)
        ListOfChoices.remove(barrier)
        ListOfHurdles.append(barrier)
    print("\nBarriers List: ", ListOfHurdles, "\n")

    for eachNode in ListOfHurdles:
        Maze[eachNode] =  "♦"   # barrier character encoding
    maze_display()


def outcome(algorithm, visited_nodes, sp):  # display the shortest path, visited nodes and time to process
    print("~" * 50, "\n" + str(algorithm).center(50) + "\n", "~" * 50, )
    print("\nVisited Nodes: ", visited_nodes)
    print("\nTime to Process and search the goal: ", len(visited_nodes), "minutes")
    print("\nShortest path: ", sp, "\n")
    maze_display()
    print("\n\n")


def maze_display():       #display the generated maze
    line = ""
    for row in range(6):
        for column in range(6):
            line = line + " " + str(Maze[row + column * 6]).center(2) + " "
        print(line)
        line = ""
        print(line)


# User Interface
start = 'yes'
while start == 'yes':
    print('~~' * 25)
    print('\033[1m' + "\tGENERATOR OF AI SEARCH ALGORITHMS" + '\033[0m')
    print('~~' * 25)

    print("1. Random Maze Generator ")
    print("2. Exit ")
    choice = int(input("\nEnter choice number [ 1/2 ] :"))
    if choice == 1:
        maze()
        DFS_algo()
        print("Chebyshev Heuristic cost:", chebyshev_distance_heuristic(S_Node, G_Node))
        AStar_Search_algo()  

    if choice == 2:
        break

    start = input('\nDo you want to Run Again? [Yes/No]: ')
    

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
	GENERATOR OF AI SEARCH ALGORITHMS
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1. Random Maze Generator 
2. Exit 

Enter choice number [ 1/2 ] :1

Start Node:  7  Goal Node:  35

Barriers List:  [18, 2, 16, 12] 

 0   6   ♦   ♦   24  30 

 1   7   13  19  25  31 

 ♦   8   14  20  26  32 

 3   9   15  21  27  33 

 4   10  ♦   22  28  34 

 5   11  17  23  29  35 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
                Depth First Search                
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Visited Nodes:  [7, 1, 0, 6, 8, 9, 3, 4, 5, 11, 10, 17, 23, 22, 21, 15, 14, 13, 19, 20, 26, 25, 24, 30, 31, 32, 33, 27, 28, 29, 35]

Time to Process and search the goal:  31 minutes

Shortest path:  [7, 8, 9, 3, 4, 5, 11, 17, 23, 22, 21, 15, 14, 13, 19, 20, 26, 25, 24, 30, 31, 32, 33, 27, 28, 29, 35] 

 0   6   ♦   ♦   24  30 

 1   7   13  19  25  31 

 ♦   8   14  20  26  32 

 3   9   15  21  27  33 

 4   10  